In [ ]:
from sklearn.metrics import log_loss
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, mean_squared_error

'''알고리즘 관련 라이브러리'''
import lightgbm as lgb

'''텐서플로 및 케라스 관련 라이브러리'''
import tensorflow as tf
import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Dropout
from keras.layers import BatchNormalization, Input, Lambda
from keras import regularizers
from k

path = 'C:/Users/kt NexR/PycharmProjects/Recommendation-algorithms/Classic_recommendation_algorithms/collaborative-filtering/data'
ratings_df = pd.read_csv(path + '/ratings.csv', encoding='utf-8')

print(ratings_df.shape)
print(ratings_df.head())

